In [7]:
import os
os.environ["OPENAI_API_KEY"] = 'your api key'

In [21]:
from langchain import PromptTemplate, OpenAI, LLMChain

prompt_template = "What is a good name for a company that makes {product}?"

llm = OpenAI(temperature=0)
llm_chain = LLMChain(
    llm=llm,
    prompt=PromptTemplate.from_template(prompt_template)
)
llm_chain("colorful socks")

{'product': 'colorful socks', 'text': '\n\nSocktastic!'}

In [10]:
llm_chain.run("colorful socks")

'\n\nSocktastic!'

In [22]:
input_list = [
    {"product": "socks"},
    {"product": "computer"},
    {"product": "shoes"}
]

text_output = llm_chain.apply(input_list)

In [13]:
print(type(text_output))

<class 'list'>


In [14]:
print(text_output)

[{'text': '\n\nCozy Toes Socks.'}, {'text': '\n\nTechCore Solutions.'}, {'text': '\n\nFootwear Factory.'}]


In [16]:
print(text_output[1])

{'text': '\n\nTechCore Solutions.'}


In [23]:
llm_chain.generate(input_list)

LLMResult(generations=[[Generation(text='\n\nSocktastic!', generation_info={'finish_reason': 'stop', 'logprobs': None})], [Generation(text='\n\nTechCore Solutions.', generation_info={'finish_reason': 'stop', 'logprobs': None})], [Generation(text='\n\nFootwear Factory.', generation_info={'finish_reason': 'stop', 'logprobs': None})]], llm_output={'token_usage': {'prompt_tokens': 36, 'total_tokens': 55, 'completion_tokens': 19}, 'model_name': 'text-davinci-003'})

In [24]:
llm_chain.predict(product="colorful socks")

'\n\nSocktastic!'

In [25]:
template = """Tell me a {adjective} joke about {subject}."""
prompt = PromptTemplate(template=template, input_variables=["adjective", "subject"])
llm_chain = LLMChain(prompt=prompt, llm=OpenAI(temperature=0))

llm_chain.predict(adjective="sad", subject="ducks")

'\n\nQ: What did the duck say when his friend died?\nA: Quack, quack, goodbye.'

In [27]:
from langchain.output_parsers import CommaSeparatedListOutputParser

output_parser = CommaSeparatedListOutputParser()
template = """List all the colors in a rainbow"""
prompt = PromptTemplate(template=template, input_variables=[], output_parser=output_parser)
llm_chain = LLMChain(prompt=prompt, llm=llm)

text_output1 = llm_chain.predict()
print(text_output1)
print(type(text_output1))



Red, orange, yellow, green, blue, indigo, violet
<class 'str'>


In [30]:
text_output2 = llm_chain.predict_and_parse()
print(text_output2)
print(type(text_output2))

['Red', 'orange', 'yellow', 'green', 'blue', 'indigo', 'violet']
<class 'list'>


In [31]:
template = """Tell me a {adjective} joke about {subject}."""
llm_chain = LLMChain.from_string(llm=llm, template=template)
llm_chain.predict(adjective="sad", subject="ducks")

'\n\nQ: What did the duck say when his friend died?\nA: Quack, quack, goodbye.'

### Question Answering

In [36]:
!pip install chromadb

  Using cached chromadb-0.3.26-py3-none-any.whl (123 kB)
     ---------------------------------------- 9.8/9.8 MB 3.3 MB/s eta 0:00:00
     -------------------------------------- 225.7/225.7 kB 4.6 MB/s eta 0:00:00
  Using cached posthog-3.0.1-py2.py3-none-any.whl (37 kB)
  Using cached tokenizers-0.13.3-cp310-cp310-win_amd64.whl (3.5 MB)
  Using cached uvicorn-0.22.0-py3-none-any.whl (58 kB)
  Using cached hnswlib-0.7.0.tar.gz (33 kB)
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Preparing metadata (pyproject.toml): started
  Preparing metadata (pyproject.toml): finished with status 'done'
     ---------------------------------------- 57.0/57.0 kB ? eta 0:00:00
  Using cached pulsar_client-3.2.0-cp310-cp310-win_amd64.whl (3.4 MB)
  Using cached onnxruntime-1.15.0-cp310-cp310-win_amd64.whl (6.7 MB)
  Using cached ove

In [33]:
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.text_splitter import CharacterTextSplitter
from langchain.vectorstores import Chroma
from langchain.docstore.document import Document
from langchain.prompts import PromptTemplate
from langchain.indexes.vectorstore import VectorstoreIndexCreator

In [34]:
with open("./state_of_the_union.txt") as f:
    state_of_the_union = f.read()
text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
texts = text_splitter.split_text(state_of_the_union)

embeddings = OpenAIEmbeddings()

Created a chunk of size 1163, which is longer than the specified 1000
Created a chunk of size 1015, which is longer than the specified 1000


In [37]:
docsearch = Chroma.from_texts(texts, embeddings, metadatas=[{"source": str(i)} for i in range(len(texts))]).as_retriever()

Retrying langchain.embeddings.openai.embed_with_retry.<locals>._embed_with_retry in 4.0 seconds as it raised APIConnectionError: Error communicating with OpenAI: ('Connection aborted.', RemoteDisconnected('Remote end closed connection without response')).


In [38]:
query = "What did the president say about Justice Breyer"
docs = docsearch.get_relevant_documents(query)

In [39]:
from langchain.chains.question_answering import load_qa_chain
from langchain.llms import OpenAI

In [40]:
chain = load_qa_chain(OpenAI(temperature=0), chain_type="stuff")
query = "What did the president say about Justice Breyer"
chain.run(input_documents=docs, question=query)

' The president did not mention Justice Breyer in this context.'

### Stuff chain

In [41]:
chain = load_qa_chain(OpenAI(temperature=0), chain_type="stuff")

In [42]:
query = "Who is the vice president"
chain({"input_documents": docs, "question": query}, return_only_outputs=True)

{'output_text': ' Joe Biden'}

### Custom Prompt

In [43]:
prompt_template = """Use the following pieces of context to answer the question at the end. If you don't know the answer, just say that you don't know, don't try to make up an answer.

{context}

Question: {question}
Answer in Italian:"""
PROMPT = PromptTemplate(
    template=prompt_template, input_variables=["context", "question"]
)
chain = load_qa_chain(OpenAI(temperature=0), chain_type="stuff", prompt=PROMPT)
chain({"input_documents": docs, "question": query}, return_only_outputs=True)

{'output_text': ' Il vice presidente è Joe Biden.'}

### Map_reduce chain

In [44]:
chain = load_qa_chain(OpenAI(temperature=0), chain_type="map_reduce")

In [45]:
query = "What about recession in America"
chain({"input_documents": docs, "question": query}, return_only_outputs=True)

{'output_text': ' The president mentioned that the American people have felt the pain of recession.'}

### Intermediate Steps

In [46]:
chain = load_qa_chain(OpenAI(temperature=0), chain_type="map_reduce", return_map_steps=True)

In [47]:
chain({"input_documents": docs, "question": query}, return_only_outputs=True)

{'intermediate_steps': [' None',
  ' "And they have done so in the midst of war and depression; at moments of great strife and great struggle."',
  " It is because of this spirit, this great decency and great strength that I have never been more hopeful about America's future than I am tonight. Despite our hardships, our union is strong. We do not give up. We do not quit. We do not allow fear or division to break our spirit. In this new decade, it's time the American people get a government that matches their decency, that embodies their strength.",
  ' "We have felt the pain of recession"'],
 'output_text': ' The president mentioned that the American people have felt the pain of recession.'}

### Custom Prompt

In [48]:
question_prompt_template = """Use the following portion of a long document to see if any of the text is relevant to answer the question. 
Return any relevant text translated into Hindi.
{context}
Question: {question}
Relevant text, if any, in Hindi:"""
QUESTION_PROMPT = PromptTemplate(
    template=question_prompt_template, input_variables=["context", "question"]
)

combine_prompt_template = """Given the following extracted parts of a long document and a question, create a final answer Hindi. 
If you don't know the answer, just say that you don't know. Don't try to make up an answer.

QUESTION: {question}
=========
{summaries}
=========
Answer in Hindi:"""
COMBINE_PROMPT = PromptTemplate(
    template=combine_prompt_template, input_variables=["summaries", "question"]
)
chain = load_qa_chain(OpenAI(temperature=0), chain_type="map_reduce", return_map_steps=True, question_prompt=QUESTION_PROMPT, combine_prompt=COMBINE_PROMPT)
chain({"input_documents": docs, "question": query}, return_only_outputs=True)

{'intermediate_steps': [' अमेरिका में मंडली के बारे में क्या?',
  ' हमारे संविधान में कहा गया है कि कई बार राष्ट्रपति को संसद को हमारे समूह के बारे में जानकारी देनी चाहिए। 220 सालों से हमारे नेताओं ने इस कर्तव्य को पूरा किया है। वे सुख और शांत',
  ' अमेरिका में अवसर के बारे में क्या? हम सभी ने बैंक बैलट को नफरत की थी। मैं इसे नफरत करता था। आप इसे नफरत करते थे। यह एक दांत की तरह की प्रतिक्रिया के समान था।',
  ' अमेरिका में अर्थव्यवस्था में क्या हुआ? उसके लिए संबंधित पाठ, यदि कोई हो, हिंदी में: अमेरिका में अर्थव्यवस्था में क्या हुआ? उसके लिए उन कई लोगों को जो इस साल सभी देश म'],
 'output_text': ' अमेरिका में अर्थव्यवस्था में क्या हुआ? अमेरिका में पिछले कुछ सालों से मंडली का अभाव हुआ है। यह अमेरिका के अर्थव्यवस्था को अस्थिर बनाने के लिए काफी प्रभावी रहा है। अम'}

### Refine

In [49]:
chain = load_qa_chain(OpenAI(temperature=0), chain_type="refine")

In [50]:
query = "What are the difficulty face by Americans's"
chain({"input_documents": docs, "question": query}, return_only_outputs=True)

{'output_text': '\n\nAmericans face difficulty with civil rights violations, employment discrimination, hate crimes, unequal pay for women, a broken immigration system, a lack of economic opportunity for many communities, and the effects of the economic crisis on the banking system, as well as the political setbacks that have been experienced this year. Despite these difficulties, Americans remain resilient and determined, and continue to demonstrate their fundamental decency and optimism.'}

### Intermediate steps

In [51]:
chain = load_qa_chain(OpenAI(temperature=0), chain_type="refine", return_refine_steps=True)


In [52]:
chain({"input_documents": docs, "question": query}, return_only_outputs=True)

{'intermediate_steps': ['\nAmericans face difficulty with civil rights violations, employment discrimination, hate crimes, unequal pay for women, and a broken immigration system.',
  '\n\nAmericans face difficulty with civil rights violations, employment discrimination, hate crimes, unequal pay for women, a broken immigration system, and a lack of economic opportunity for many communities.',
  '\n\nAmericans face difficulty with civil rights violations, employment discrimination, hate crimes, unequal pay for women, a broken immigration system, a lack of economic opportunity for many communities, and the effects of the economic crisis on the banking system.',
  '\n\nAmericans face difficulty with civil rights violations, employment discrimination, hate crimes, unequal pay for women, a broken immigration system, a lack of economic opportunity for many communities, and the effects of the economic crisis on the banking system, as well as the political setbacks that have been experienced th

### Custom Prompt

In [53]:
refine_prompt_template = (
    "The original question is as follows: {question}\n"
    "We have provided an existing answer: {existing_answer}\n"
    "We have the opportunity to refine the existing answer"
    "(only if needed) with some more context below.\n"
    "------------\n"
    "{context_str}\n"
    "------------\n"
    "Given the new context, refine the original answer to better "
    "answer the question. "
    "If the context isn't useful, return the original answer. Reply in Italian."
)
refine_prompt = PromptTemplate(
    input_variables=["question", "existing_answer", "context_str"],
    template=refine_prompt_template,
)


initial_qa_template = (
    "Context information is below. \n"
    "---------------------\n"
    "{context_str}"
    "\n---------------------\n"
    "Given the context information and not prior knowledge, "
    "answer the question: {question}\nYour answer should be in Italian.\n"
)
initial_qa_prompt = PromptTemplate(
    input_variables=["context_str", "question"], template=initial_qa_template
)
chain = load_qa_chain(OpenAI(temperature=0), chain_type="refine", return_refine_steps=True,
                     question_prompt=initial_qa_prompt, refine_prompt=refine_prompt)
chain({"input_documents": docs, "question": query}, return_only_outputs=True)

{'intermediate_steps': ['\nGli americani affrontano difficoltà come la discriminazione nei luoghi di lavoro, la mancanza di diritti civili, la mancanza di parità di retribuzione tra uomini e donne, e la mancanza di un sistema di immigrazione equo.',
  '\n\nGli americani affrontano difficoltà come la discriminazione nei luoghi di lavoro, la mancanza di diritti civili, la mancanza di parità di retribuzione tra uomini e donne, la mancanza di un sistema di immigrazione equo e la disuguaglianza economica.',
  '\n\nGli americani affrontano difficoltà come la discriminazione nei luoghi di lavoro, la mancanza di diritti civili, la mancanza di parità di retribuzione tra uomini e donne, la mancanza di un sistema di immigrazione equo, la disuguaglianza economica e la crisi economica.',
  '\n\nGli americani affrontano difficoltà come la discriminazione nei luoghi di lavoro, la mancanza di diritti civili, la mancanza di parità di retribuzione tra uomini e donne, la mancanza di un sistema di immigra

### Map Rerank

In [54]:
chain = load_qa_chain(OpenAI(temperature=0), chain_type="map_rerank", return_intermediate_steps=True)

In [55]:
query = "What are the difficulty face by Americans's"
results = chain({"input_documents": docs, "question": query}, return_only_outputs=True)

In [56]:
results["output_text"]

' Difficulty faced by Americans include civil rights violations, employment discrimination, unequal pay for women, and a broken immigration system.'

In [57]:
results["intermediate_steps"]

[{'answer': ' Difficulty faced by Americans include civil rights violations, employment discrimination, unequal pay for women, and a broken immigration system.',
  'score': '100'},
 {'answer': ' This document does not answer the question.', 'score': '0'},
 {'answer': ' The difficulty faced by Americans is the bank bailout, which was unpopular with both Democrats and Republicans.',
  'score': '80'},
 {'answer': ' The difficulty faced by Americans include political setbacks, economic recession, and other struggles.',
  'score': '80'}]

### Custom Prompt

In [60]:
from langchain.output_parsers import RegexParser

output_parser = RegexParser(
    regex=r"(.*?)\nScore: (.*)",
    output_keys=["answer", "score"],
)

prompt_template = """Use the following pieces of context to answer the question at the end. If you don't know the answer, just say that you don't know, don't try to make up an answer.

In addition to giving an answer, also return a score of how fully it answered the user's question. This should be in the following format:

Question: [question here]
Helpful Answer In Hindi: [answer here]
Score: [score between 0 and 100]

Begin!

Context:
---------
{context}
---------
Question: {question}
Helpful Answer In Hindi:"""
PROMPT = PromptTemplate(
    template=prompt_template,
    input_variables=["context", "question"],
    output_parser=output_parser,
)

chain = load_qa_chain(OpenAI(temperature=0), chain_type="map_rerank", return_intermediate_steps=True, prompt=PROMPT)
query = "Who is the vice president"
chain({"input_documents": docs, "question": query}, return_only_outputs=True)

{'intermediate_steps': [{'answer': ' उम्मीदवार कौन है?', 'score': '0'},
  {'answer': ' उम्मीदवार बाइडेन है।', 'score': '100'},
  {'answer': ' उम्मीद करता हूँ कि आपको यह जानना आवश्यक नहीं है।',
   'score': '0'},
  {'answer': ' उम्मीदवार कौन है?', 'score': '0'}],
 'output_text': ' उम्मीदवार बाइडेन है।'}